# Data Visualization in R

Julia Lane, Brian Kim, Benjamin Feder, Maryah Garner, Ekaterina Levitskaya, Allison Nunez, Rukhshan Arif Mian. </center>

## Introduction

This notebook covers how to use different visualization methods in R to further explore and analyze data. The goal of this notebook is twofold:

1. Allowing you to understand the data better
2. Enabling you to clearly and accurately communicate results using labels and various plotting adjustments while also keeping in mind disclosure review processes

We discuss the following types of visualizations: 
- **Histogram**: visualizing distributions, continuous variables
- **Bar plot**: visualizing relationships between numerical and categorical variables
- **Stacked Bar plots**: looking at numeric values across one categorical variable to two.
- **Waffle Plot**: visualizing data in relation to its population
- **Heatmap** : adding highlights to the data with color-coding


### Motivation

In answering a series of specific questions about the data, various visualizations are introduced to provide a clearer view of the data than just using summary statistics. These powerful graphics may better convey conclusions from an analysis compared to the previously introduced summary statistics in the Data Exploration notebook.

The questions covered in this notebook are:
- What is the distribution of semesters funded for doctorate students?
- What are the most common sources of funding? How can we visualize sources of funding and semesters funded by each source effectively?
- How does coverage vary by sex in the SED data?
- How does coverage vary in SED compared to UMETRICS?


### R Setup

As always, the code for this notebook starts by importing necessary R libraries, setting the viewing options, and connecting to the database.

In [ ]:
options(warn=-1)

#database interaction imports
suppressMessages(library(odbc))

# for data manipulation/visualization
suppressMessages(library(tidyverse))

# for date-time manipulation
suppressMessages(library(lubridate))

# for calculating percentages
suppressMessages(library(scales))

# for waffle plot
suppressMessages(library(waffle))

options(warn=0)

In order to ease the viewing of graphs, we run the following:

In [ ]:
theme_set(theme_gray(base_size = 24))

# Adjust repr.plot.width and repr.plot.height to change the size of graphs
options(repr.plot.width = 20, repr.plot.height = 12)

Next, we connect to the database

In [ ]:
# Connect to the database
con <- DBI::dbConnect(odbc::odbc(),
                     Driver = "SQL Server",
                     Server = "msssql01.c7bdq4o2yhxo.us-gov-west-1.rds.amazonaws.com",
                     Trusted_Connection = "True")

## Reading in previously created CSVs

It is good practice to call all the data we would like to use (be it through CSVs or SQL) beforehand instead of calling them later on.

In [ ]:
# reading in the CSV for source of support for doctorate students who were in SED-UMETRICS
source_umetrics_df <- read_csv("Tables\\Source_of_support_UMETRICS.csv", col_types = cols())

# reading in the distribution of source of support for doctorate students in SED
source_sed_df <- read_csv("Tables\\primary_support_coverage.csv", col_types = cols())

# reading in top 3 sources of support for doctorate students in SED-UMETRICS
top3_fund_srce <- read_csv("Tables\\top3_by_sem.csv", col_types = cols())


## Read in tables from SQL

In [ ]:
qry <- '
select DISTINCT(phdinst)
FROM tr_ncses_2021.dbo.umetrics_inst_xwalk
'
umetrics_institutions <- dbGetQuery(con, qry)

In [ ]:
# Left join between SED table and UMETRICS semester table
qry <- " 
select sed.drf_id, sem.semester, sed.srceprim, sed.phdinst, sem.any_federal, sem.any_non_federal, sed.sex
from ds_nsf_ncses.dbo.nsf_sed sed 
left join tr_ncses_2021.dbo.sed_umetrics_xwalk xwalk
on xwalk.drf_id = sed.drf_id
left join ds_iris_umetrics.dbo.semester sem
on sem.emp_number = xwalk.emp_number
where sed.phdfy = '2015'
"
sed_umetrics <- dbGetQuery(con, qry)

## Getting started with visualization in R:

For most visualizations in this notebook, the R package`ggplot2` is used (`ggplot2` is included in the `tidyverse` suite of packages). The syntax of `ggplot2` in most cases is consistent:

- start with `ggplot()`
- then, supply a dataset and aesthetic mapping with `x` pertaining to the variable on the x-axis, and so on, for example: `ggplot(dataset, aes(x = ..., y = ...)`
- from there, provide a geometric object represented by `geom_` to convey the desired type of visualization
- finally, add additional layers if necessary using `+`
 
For example, the code below can be used

    ggplot(data, aes(x = ...) + 
        geom_histogram()
    
to create a histogram (geometric object), or with the addition of an extra layer

    ggplot(data, aes(x = ...) + 
        geom_histogram() + 
        labs(title = 'My plot title')
        
to create a histogram with a title for the graph, and so on. </center>

`ggplot()` can work together with other tidy functions, as `%>%` can be used to set the desired data set as the `data` argument in the `ggplot()` call. (Wickham 2016). The syntax for this would be:
```
data %>%
    ggplot(aes(x = ...)) + 
    geom_histogram() + 
    labs(title = 'My plot title')
 ```

> Note: The `aes()` call can contain additional arguments outside of `x` and `y` to potentially match the `fill`, `color`, `linetype`, and additional specifications of specific variables in a dataset.


## Understanding the data better

### Histogram:

As mentioned above, our first goal is to focus on visualization methods that aid our understanding of the data we are using. Here, our objective is to just think of methods that help us explore the data we have.

#### UMETRICS Semester Funded
In this section, we first look at a histogram – these allow us to see the distribution of numerical data. Recall from the measurement notebook, we created a table that allowed us to see the number of semesters funded for doctorate students. In order to further understand this information, we plot a histogram. It will allow us to see the counts of the number of semesters funded for doctorate students.

We use the variable, **sem_funded** from `source_umetrics_df` (created in the measurement notebook) as the numeric variable we would like to visualize. The code for this is given below:

In [ ]:
# calling the data frame
source_umetrics_df %>% 
    # calling ggplot and defining the aesthetic (since we want to visualize sem_funded, we use that as our x variable)
    ggplot(aes((x=sem_funded))) + # using a plus sign to add the histogram layer
    geom_histogram(bins=20) # grouping data into "bins" of equal width. Each bin is plotted as a bar whose height corresponds to how many data points are in that bin. 

In the histogram above, we observe a count for each of the various number of semesters (funded) in our data. When working on your research project, it is encouraged that you utilize a histogram to visualize a numeric variable's distributions.
> In some cases, semesters funded can be a categorical variable, such as when each value corresponds to a specific range, i.e. <4, 5-6, 7-8, etc., but that is not the case here. We do share an example later in this notebook.

### Bar Graph: Graphing the Distribution of a Categorical Variable

#### Source of Support

Histograms are useful to develop and enhance our understanding of numerical variables. In this section, we introduce the Bar Graph – these allow us to present categorical data with rectangular bars with heights or lengths proportional to the values that they represent. The syntax for plotting a Bar Graph is slightly different from that required for a histogram – in this case, we use `geom_bar` and we add a parameter `stat`.


In terms of a categorical variable, we consider `source_cat` in the `source_umetrics_df` data frame. `source_cat` corresponds to the source of funding for SED doctorate students who graduated in 2015 and are in the UMETRICS data.

In [ ]:
bar_graph_source <- source_umetrics_df %>% 
    ggplot(aes(x=source_cat)) + # calling ggplot and defining our x variable to be source_cat
    geom_bar(stat='count') # adding a layer for geom_bar and calling stat. we equate this to "count" to count the number of observations for source_cat.

# calling the stored bar graph variable
bar_graph_source

#### Grouped Bar Graph: Number of semesters funded by source of support

At times during your research, you may be interested in looking at how distributions vary by groups. An example that we may want to consider is that how does the distribution of number of semesters funded vary by source of support? In terms of variables, how would `sem_funded` vary by `source_cat` in the `source_umetrics_df` data frame. We combine `geom_bar` with `facet_wrap` (performs the grouping) to create this.

Note that we are using **sem_funded** with `geom_bar` and not `geom_histogram` – this is to show that we can plot counts of numerical variables using bar graphs as well (`geom_bar` assumes that the variable provided is categorical). The code is as follows:

In [ ]:
facet_wrap_bar <- source_umetrics_df %>% # call the relevant data frame
    ggplot(aes(x=sem_funded)) + # define x as sem_funded 
    geom_bar(stat='count') + # stat = 'count' in order to count the number of observations
    facet_wrap((~source_cat)) # add a layer for facet_wrap with the variable want to group by (~ sign is needed here)

# calling saved plot
facet_wrap_bar

The grouped bar graphs allow us to see the counts of number of semesters funded for each source of support category. This provides us with a basic overview of what our data frame looks like. Understandably however, the grouped chart is difficult to interpret and may not be the best at communicating our findings. We propose an alternate way of approaching the graph later in this notebook.

Up until now, we have introduced a histogram, a bar graph and a grouped bar graph – these are all important in understanding the data. However, these might not be the most useful ways of communicating our findings effectively, nor are these necessarily disclosure friendly. In the next section, we explore visualization methods that can be used to effectively communicate key findings from our data while also thinking about methods that can help make our visualizations pass disclosure review.

## Effective Communication using visualizations

In this section, we focus on visualization methods that not only aid our understanding of the data but also make it easier for us to communicate our findings. Furthermore, we look at how you may approach visualizations for the purpose of your research project. Lastly, we draw attention to best practices in terms of disclosure review processes for data visualizations – we will do this by incorporating the following in all graphics we create:

1. Adding the data source 
2. Adding a Title to every plot
3. Using readable labels (in terms of font size and orientation) for axes and legends
4. Censoring any counts that are disclosive

### Defining a colorblind friendly palette

As colors play a key role in data visualizations, our goal should be to make our graphics as accessible as we can. For the purpose of our visualizations in this section, we introduce a palette considered to be colorblind friendly. 

In [ ]:
# introduce colorblind friendly palette: mustard-orange, blue, green, yellow, orange, light purple, grey
cbPalette <- c("#E69F00", "#56B4E9", "#009E73", "#F0E442", "#0072B2", "#D55E00", "#CC79A7", "#999999")

### Grouped Bar Graph

This section's first visualization builds on the bar graph that we implemented in the previous section. Here, we introduce a different implementation of a grouped bar graph.



In [ ]:
# save a new data frame
sed_coverage_by_sex <- sed_umetrics %>%
    # create sex_str
    mutate(sex_str = 
           case_when(sex == 1 ~ "Male",
                     sex == 2 ~ "Female"
        )) %>%
    # join with source_sed_df to bring in recoded categories
    inner_join(source_sed_df, by.x='srceprim', by.y='source_name') %>%
    # group by newly created variable (sex_str) and source_cat
    group_by(sex_str, source_cat) %>%
    # count the number of unique observations
    summarise(n=n_distinct(drf_id))

# view the first few observations
head(sed_coverage_by_sex)

In [ ]:
grp_bar_source_of_support <- sed_coverage_by_sex %>%
    ggplot(aes(x=source_cat, # recoded source of support category
               y=n,  # refers to count
               fill=sex_str) # refers to the category we want to fill in the bars by (in this case, sex_str)
            ) +
    geom_bar(stat = "identity", # we use stat = "identity" to use the y (n) values directly.
             position = position_dodge() # stacking the bars side by side
            ) + 
    theme(axis.text.x = element_text(angle = 90)) + # rotating text labels
    scale_fill_manual(values=cbPalette) + # adding the color-blind friendly palette 
    labs(
        title = "", # defining title
        y = 'Count of graduate students', # labelling y axis 
        x = 'Source of Support', # labelling x axis
        caption = 'Source: 2014-2017 SED-UMETRICS Linked Data', # specifiying the data source
        fill = "Sex") 

grp_bar_source_of_support

### Stacked Bar Graph

Stacked bar graphs are another way of visualizing multiple variables together. For the purpose of this sub-section, we think about how we can look at number of semesters funded by whether or not someone received federal funding (or both) – we will use the `sed_umetrics` data frame for this. 

Before delving into the visualization itself, we need to perform some data manipulation tasks:

1. Create a categorical variable that specifies if someone (at any point during their PhD) received federal funding, non-federal funding or both at any point
2. Further group the variable associated with number of semesters funded. Currently, this variable takes up several values and our goal is to group these values into smaller categories – this would allow for a cleaner, more readable visualization. Recall that in some cases semesters funded can be a categorical variable, such as when each value corresponds to a specific range. In this case, we will define 3 categories: 
    - 1-3
    - 4-7
    - \> 7


In [ ]:
fed_flag_df <- sed_umetrics %>%
    group_by(drf_id) %>%
    mutate(max_fed = max(any_federal), max_nonfed = max(any_non_federal)) %>%
    mutate(fed_flag = 
           case_when(max_fed == 1 & max_nonfed == 0 ~ "Only Federal",
                     max_fed == 1 & max_nonfed == 1 ~ "Both Federal and Non Federal",
                     max_fed == 0 & max_nonfed == 1 ~ "Only Non Federal",
                     max_fed == 0 & max_nonfed == 0 ~ "No Funding"
    )) %>%
    distinct(drf_id, .keep_all=TRUE) %>%
    select(drf_id, fed_flag)

In [ ]:
source_umetrics_df_joined <- source_umetrics_df %>%
    # creating a categorical variable for number of semesters_funded
    mutate(sem_cat = 
            case_when(sem_funded %in% c(1:3) ~ "1-3", 
                     sem_funded %in% c(4:7) ~ "4-7",
                     sem_funded > 7 ~ "greater than 7"
                     )
          ) %>%
    # joining based on drf_id to get the categorical variable associated with type of funding
    inner_join(fed_flag_df, by='drf_id')
    
head(source_umetrics_df_joined)

Now that we have a data frame with categorical variables for number of semesters funded and type of support, we plot a stacked bar graph using the following code:

In [ ]:
stacked_bar <- source_umetrics_df_joined %>%
    ggplot(aes(x=sem_cat, # our x axis shows the categorical variable we created for number of semesters funded
               fill=fed_flag #  filling the bars with our categorical variable for type of funding
              )) + 
    geom_bar() + # calling geom_bar
    scale_fill_manual(values=cbPalette) + # using a color-blind friendly palette
    labs(
        title = "", # defining the title
        y = 'Count of graduate students', # labelling the y axis
        x = 'Number of semesters of funding', # labelling the x axis
        caption = 'Source: 2014-2017 SED-UMETRICS Linked Data', # adding the name of the data source
        fill = "Type of Funding" # labelling the legend
        ) 

stacked_bar

### Grouped Line Plot
Recall that in the second data exploration notebook, `02_Data_Exploration_UMETRICS.ipynb`, we identified the top 3 sources of funding for graduate students (in SED) who graduated in 2015. We also created a table that provided us with counts associated with each source of funding over time – here, time was defined in terms of semesters. For your research project, you may be interested in learning about the distribution of other sources of funding or even variables over time. In this sub-section, we explore this idea by taking into account the top 3 sources of funding. When looking at time as a variable, it is encouraged to utilize a line (`geom_line` in R) plot as it allows for effective interpretation. 


To make our visualization easier to understand, we perform some minor data cleaning tasks:

In [ ]:
top3_fund_srce$sem_short <- gsub("jan-apr", "s1", top3_fund_srce$semester) # replace jan-apr with s1 - corresponds to semester 1
top3_fund_srce$sem_short <- gsub("may-aug", "s2", top3_fund_srce$sem_short) # replace may-aug with s2 - corresponds to semester 2
top3_fund_srce$sem_short <- gsub("sep-dec", "s3", top3_fund_srce$sem_short) # replace sep-dec with s3 - corresponds to semester 3

head(top3_fund_srce)

Now, we move towards plotting a line plot that looks at the top 3 sources of funding vary over time.

In [ ]:
grouped_line_plot <- top3_fund_srce %>%
    ggplot(aes(x=sem_short, # defining the x axis
               y=count_individuals, # defining the y axies
               group=modal_funder, # defining how we would like to group by (required for geom_line)
               color=modal_funder)) + # we use different colors for each source of funding
    geom_line(size=1) + # calling geom_line + defining the thickness of the line
    theme(axis.text.x = element_text(angle = 90)) + # rotating x labels by 90 degrees
    labs(
        title = "", # defining the title
        y = 'Count of graduate students', # labelling the y axis
        x = 'Semester-Year', # labelling the x axis
        caption = 'Source: 2014-2017 SED-UMETRICS Linked Data', # adding the name of the data source
        fill = "Source of Funding" # labelling the legend
        ) 
grouped_line_plot

### Waffle Plot

The next type of user and disclosure friendly plot we look at is a waffle chart. Such charts allow us to see progress towards a target or a completion percentage. For our purpose, we use a waffle plot to compare:
1. The percentage of doctorate students who are in the UMETRICS data
2. Percentage of students whose institutions are in the UMETRICS data but they themselves are not
3. Percentage of students who are not in the UMETRICS data because their institutions are not in it

Indeed, we can use a bar plot to compare these 3 values. However, we show that using a waffle plot might be a better way to communicate this information.

We calculate the above percentages by first performing some data manipulation tasks:

In [ ]:
# Create a new column called "umetrics_category" based on whether the "semester" variable is missing,
# and whether IPEDS ID is found in the UMETRICS institution crosswalk
sed_umetrics <- sed_umetrics %>%
    mutate(umetrics_category =
          case_when(!is.na(semester) ~ 'in_umetrics',
                    is.na(semester) & phdinst %in% umetrics_institutions$phdinst ~ 'inst_in_umetrics_student_not',
                    TRUE ~ 'not_in_umetrics_because_of_inst'))

# Save the underlying data for the waffle plot
waffle_df <- sed_umetrics %>%
    group_by(umetrics_category) %>%                  # group by "umetrics_category"
    summarize(indiv_count = n_distinct(drf_id),      # count number of unique individuals
             inst_count = n_distinct(phdinst))       # count number of unique institutions 


waffle_df

We would want to create this chart by keeping in mind disclosure review processes. In order to account for these, we perform some additional (manual) data manipulation to round the counts each count, sum it and then round the summed count. 

We create a final variabe, **perc_rounded** that we use to plot the waffle chart. 

In [ ]:
waffle_df_rounded <- waffle_df %>%
                    mutate(indiv_count_rounded = 
                            case_when(indiv_count == REDACTED ~ 3200, # round down values to two digits 
                                      indiv_count == REDACTED ~ 7300,
                                      indiv_count == REDACTED ~ 44000)
                           )
waffle_df_rounded <- waffle_df_rounded %>%
    mutate(perc_rounded = round(indiv_count_rounded/round(sum(indiv_count_rounded), 2), 2)*100)

head(waffle_df_rounded)


In [ ]:
# next, we store the rounded percentages as variables for each category to input for the waffle chart 
in_umetrics <- waffle_df_rounded$perc_rounded[1]
inst_in_umetrics_student_not <- waffle_df_rounded$perc_rounded[2]
not_in_umetrics_because_of_inst <- waffle_df_rounded$perc_rounded[3]


In [ ]:
# use the waffle command to initialize the chart:
waffle_plot <- waffle(
  c('In UMETRICS' = in_umetrics, # labelling and defining the percentage for In UMETRICS
    'Inst in UMETRICS' = inst_in_umetrics_student_not,
    'Not in UMETRICS' = not_in_umetrics_because_of_inst # labelling and defining the percentage for Not in UMETRICS
   ), 
    rows = 10, # even though this is associated with rows, think of this value as the number of columns we use in our waffle chart
    colors = c("#0072b2","#f0e442", "#d55e00"), # we use color-blind friendly colors here,
    title = '', # adding a title for the Waffle Chart
    legend_pos="bottom", # positioning the legend to appear at the bottom
    xlab="1 tile = 1%",# adding a label to define how we measure each tile
    flip=TRUE)+ # rotating the plot anti-clockwise by 90 degrees to make it easier to view
    ggtitle("Most students are not in UMETRICS because of their institution") + # defining the title of the plot
    theme(
        plot.title = element_text(hjust = .5, size = 24, face = "bold"), # changing plot title characteristics (alignment, font size and making it bold)
        legend.text = element_text(size = 20), # increasing size of the legend text
        axis.title.x = element_text(size = 25) # increaseing size of the xlabel
        )

waffle_plot

Through the waffle chart above, we are able to easily track the distribution of individuals who are in UMETRICS, individuals whose institutions are not in UMETRICS and individuals who are in UMETRICS because their institutions are not in UMETRICS. 

### Heatmap: Number of semesters funded by source of support

Recall that we created a grouped bar graph in the "Understanding the data better" section that looked at the number of semesters funded by source of support. In this sub-section, we implement an alternate way of showing the same information. Here, we use a heatmap to see the variation of number of semesters funded by each source of support category. 


Before writing out the code for creating a heatmap, we manipulate our data to have counts for each **sem_funded**-**source_cat** combination. 

In [ ]:
source_umetrics_count <- source_umetrics_df %>%
    group_by(sem_funded, source_cat) %>% # gorup by sem_funded and source_cat
    mutate(count = n_distinct(drf_id)) %>% # count the number of distinct drf_ids
    arrange(count)

head(source_umetrics_count)

Note there are many observations for the higher number os semesters funded with an insufficient munber of people to pass disclosure review. We will right censor the data, (group together counts for all senester greater then or equal to 9) to the figure will pass disclosure review. 

> To apply this right censoring we use the `mutate` and `ifelse` functions to create a new column that is equal to 9 if the number of semesters is grater than or equal to **9+** and equal to the number of semesters otherwise. We use the `ifelse` function instead of the `case_when` function becuase it is a little easier when there is only one rule that needs to be applied. 

> since we want to use **9+** which is a stirng, we also need to use the function `as.character` to 


In [ ]:
source_umetrics_count2 <- source_umetrics_df %>%
# make sem_funded2 equal to 9 if sem_funded >= 9, and equeal to sem_funded otherwise
    mutate(sem_funded2 = ifelse(sem_funded >= 9, "9+", as.character(sem_funded))) %>%  
    group_by(sem_funded2, source_cat) %>% # gorup by sem_funded and source_cat
    mutate(count = n_distinct(drf_id)) %>% # count the number of distinct drf_ids
    arrange(count)
    

Now that we have a data frame with counts for each **sem_funded**-**source_cat** combination, we use `geom_tile` to create a heatmap:

In [ ]:
heatmap_sem_funded_source <- source_umetrics_count2 %>%
    ggplot(aes(x=sem_funded2, # defining the x axis – number of semesters funded 
               y=source_cat, # defining the y axis – source of support categories
               fill=count) # filling the tiles with the count 
          ) + 
    geom_tile() + # calling geom_tile 
    scale_fill_distiller(palette = 'Blues', direction = 1) + # defining the color palette (color-blind friendly) and equating direction to 1 so tile color becomes darker as count increases 
    labs(
        title = '', # defining the title
        y = 'Source of Support Category', # labelling the y axis
        x = 'Number of semesters of funding', # labelling the x axis
        caption = 'Source: 2014-2017 SED-UMETRICS Linked Data', # adding the name of the data source
        fill = 'Count of doctorate students' # labelling the legend
        )  
heatmap_sem_funded_source

This is an alternate and recommended way of the plot we made in the "Understanding the data better" section. There are fewer moving parts in this type of visualization and thus, it makes it easier to understand and communicate. We are able to effectively point out any differences we see across different categories of source of support.

# Saving

We use `ggsave` to save any plots that we have created. The syntax is as follows:

```
ggsave(filepath, plot_name, width, height)
```

There are additional parameters that you may utilize – you are encouraged to utilize `?ggsave` for this purpose. 

In [ ]:
# saving heatmap
ggsave('Figures\\heatmap.png', heatmap_sem_funded_source, width=12, height=6)

# saving waffle plot
ggsave('Figures\\waffle_plot.png', waffle_plot, width=10, height=10)

# saving stacked bar
ggsave('Figures\\stacked_bar.png', stacked_bar, width=10, height=10)

## References

Wickham H (2016). _ggplot2: Elegant Graphics for Data Analysis._ Springer-Verlag New York. ISBN 978-3-319-24277-4, https://ggplot2.tidyverse.org